In [ ]:
import lxml.html
import lxml.etree
import requests
from sqlalchemy import func, create_engine
from sqlalchemy import Column, Integer, Text, DateTime
from sqlalchemy.schema import Index
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base


In [ ]:
Base = declarative_base()

class Question(Base):
    __tablename__ = 'question'

    id = Column(Integer, primary_key=True)
    text = Column(Text, nullable=False)
    answer = Column(Text, nullable=False)
    
    def __init__(self, text, answer):
        self.text = text
        self.answer = answer


def init_db(db_url):
    engine = create_engine(db_url)
    Base.metadata.bind = engine
    Base.metadata.create_all()
    return sessionmaker(bind=engine)

db_session = init_db('sqlite:///chgk.db')()



In [ ]:
def followTheLink(link):
    url = 'https://db.chgk.info/' + link
    response = requests.get(url)
    return lxml.html.fromstring(response.text)

In [ ]:
response = requests.get('https://db.chgk.info')
tree = lxml.html.fromstring(response.text)

nextPageIsExist = True

while nextPageIsExist == True:
    urles = tree.xpath('//table[@class="last_packages"]//td[2]/a/@href')
    for url in urles:
        roundTree = followTheLink(url)
        questions = roundTree.xpath('//div[@class="question"]')
        
        for i, q in enumerate(questions):
            text = ''.join(q.xpath('p/text()')).replace('\n', '')
            answer = ''.join(q.xpath('.//p/strong[@class="Answer"]/../text()')).replace('/n', '')
            newQuestion = Question(text, answer)
            db_session.add(newQuestion)
        db_session.commit()
            
    nextPage = tree.xpath('//li[@class="pager-next"]')
    if nextPage:
        tree = followTheLink(nextPage[0].xpath('a/@href')[0])
    else:
        nextPageIsExist = False
    